## Market Basket Analysis
Using transaction level data, retailers want to understand more about consumer purchases. In this analysis, we will be focusing on if there are products that are more likely to be purchased together. Understanding this has many useful applications for the retailer, such as product placement, potential promotion offers, and other cross-sell opportunities. 

### Question: Are there products that are more likely to be purchased together? 


In [1]:
# import packages
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\Nicole Eberle\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Import and Format Data

In [3]:
# import data
POSData = pd.read_excel('Market Basket Analysis Data.xlsx', sheet_name='POS Transaction Data')
POSData.head(5)

C:\Users\Nicole Eberle\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Timestamp,Transaction Number,Product UPC Number,Price,Retailer,Unnamed: 5
0,2022-12-02 08:15:50.000,123098,111100001234,1.80,Fast Mart,NaN
1,2022-12-02 08:15:50.000,123098,111100001235,23.45,Fast Mart,NaN
2,2022-12-02 08:15:50.000,123098,111100001236,6.99,Fast Mart,NaN
3,2022-12-02 08:19:51.920,123099,111100001236,6.99,Quick Stop,NaN
4,2022-12-02 08:19:51.920,123099,111100001237,7.15,Quick Stop,NaN


In [4]:
ProductKey = pd.read_excel('Market Basket Analysis Data.xlsx', sheet_name='Product UPC Key')
ProductKey.head(5)

,Product UPC Number,Product Name
0,111100001234,Coke 20 oz
1,111100001235,Miller Lite 24 Pack
2,111100001236,Pepsi 12 Pack
3,111100001237,Coke 12 Pack
4,111100001238,Doritos 12 oz.


In [5]:
#drop extra column
POSData = POSData.drop('Unnamed: 5', axis=1)

# make transaction number a string in order to concatenate
POSData['Transaction Number'] = POSData['Transaction Number'].astype(str)

# create unique retailer-transaction number (while transaction numbers will be unique at each retailer, they may not be unique across various retailers/p)
POSData['RetailerTransactionNumber'] = POSData.Retailer.str[:1] + '-' + POSData['Transaction Number']

# merge data sets to bring in product name
POSData = POSData.merge(ProductKey, 
                      on ='Product UPC Number', 
                      how ='left')

# remove space from product name to avoid future issues
POSData.rename(columns = {'Product Name':'ProductName'}, inplace = True)

# keep only RetailerTransactionNumber and ProductName columns
POSData = POSData[['RetailerTransactionNumber','ProductName']]

In [6]:
POSData.head(5)

,RetailerTransactionNumber,ProductName
0,F-123098,Coke 20 oz
1,F-123098,Miller Lite 24 Pack
2,F-123098,Pepsi 12 Pack
3,Q-123099,Pepsi 12 Pack
4,Q-123099,Coke 12 Pack


In [7]:
# create a list of products purchased for each individual transaction
Transactions = POSData.groupby('RetailerTransactionNumber').apply(lambda x: set(x['ProductName'])).tolist()

In [8]:
# use the transaction encoder to format transactions into matrix for further analysis
te = TransactionEncoder()
te_ary = te.fit(Transactions).transform(Transactions)
Basket = pd.DataFrame(te_ary, columns=te.columns_)
Basket 

,Bud Light 24 Pack,Coke 12 Pack,Coke 20 oz,Doritos 12 oz.,Hersheys Candy,Lays Chips 12 oz.,M&M's Candy,Miller Lite 24 Pack,Pepsi 12 Pack,Pepsi 20 oz,Slim Jim,Starbucks Ice
0,False,False,True,False,False,False,False,True,True,False,False,False
1,False,False,True,False,False,True,False,False,False,False,True,False
2,True,False,False,True,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,True,True,False,False
4,True,True,False,False,True,True,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
756,False,False,False,False,False,False,False,False,False,False,False,True
757,False,False,False,False,False,False,False,False,False,False,True,False
758,False,False,False,False,False,False,False,False,False,False,True,False
759,True,False,False,True,False,False,False,False,False,False,False,True


## Frequent Itemsets

In [9]:
print(f"Out of {len(Basket)} transactions, {len(Basket[(Basket > 0).sum(axis=1) >= 2])} transactions contained 2 or more items ({round((len(Basket[(Basket > 0).sum(axis=1) >= 2])/len(Basket))*100,2)}%).")

Out of 761 transactions, 322 transactions contained 2 or more items (42.31%).


Transactions with only one item will not help us determine if there are products that are more likely to be purchased together. Therefore, it is important to remember when looking at support metrics for the rest of this analysis that 58% of transactions only contain one item.
- Support: how often an itemset appears in all purchases. This can be used to asses the overall popularity of a given itemset and is calculated as (total transactions with given itemset)/(total transactions)

In [10]:
# create table of itemsets that appear in at least 2% of transactions
frequent_itemsets = apriori(Basket, min_support=0.02, use_colnames=True)
frequent_itemsets.sort_values('support',ascending=False)

,support,itemsets
0,0.241787,(Bud Light 24 Pack)
10,0.204993,(Slim Jim)
5,0.178712,(Lays Chips 12 oz.)
4,0.173456,(Hersheys Candy)
1,0.173456,(Coke 12 Pack)
3,0.153745,(Doritos 12 oz.)
2,0.101183,(Coke 20 oz)
7,0.095926,(Miller Lite 24 Pack)
9,0.094612,(Pepsi 20 oz)
11,0.089356,(Starbucks Ice)


In the above table, the first 12 rows show each individual item and how often a transaction includes that item (Ex: 24% of all transactions contain a Bud Light 24 Pack, followed by 20% containing slim jims). After the individual items, we begin to see our items that most frequently appear together in a transaction. The top combinations are: 
1. Slim Jim & Bud Light 24 Pack (3.55% of total transactions)
2. Slim Jim & Coke 12 Pack (3.55%)
3. Lays Chips 12oz & Bud Light 24 Pack (3.42%)
4. Coke 20 oz & Bud Light 24 Pack (3.15%)
5. Doritos 12oz & Bud Light 24 Pack (3.15%)

Although 3% of total transactions containing a certain combination of products may sound low, we need to remember that only 42% of the transactions had more than one item. 

While this shows products that most frequently appear together in a transaction, in order to get a deeper understanding of these relationships, we will want to look at association rules.

### Association Rules 
Association rules help uncover patterns and rules in transactional datasets using if-then statements. The results will help understand if these rules are significant or if the items are actually independent of each other. 
- Antecedent: IF part of statement
- Consequent: THEN part of statement
- Support: Percent of total transactions that contain both the antecedent and consequent 
- Confidence: Likelihood of consequent given antecedent
- Lift: How many times more likely the consequent will occur when the antecedent is true compared to how often the consequent occurs on its own

Now we will look at rules that appear within the POS data that have a confidence of at least .10. This means that of customers who bought the antecedent, 10% also bought the consequent. 


In [11]:
Rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.10).sort_values('confidence',ascending=False)
Rules.drop(['leverage', 'conviction'], axis=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
3,(Coke 20 oz),(Bud Light 24 Pack),0.101183,0.241787,0.031537,0.311688,1.289102
11,(Miller Lite 24 Pack),(Bud Light 24 Pack),0.095926,0.241787,0.028909,0.301370,1.246426
12,(Pepsi 12 Pack),(Bud Light 24 Pack),0.085414,0.241787,0.022339,0.261538,1.081689
24,(Coke 20 oz),(Slim Jim),0.101183,0.204993,0.021025,0.207792,1.013653
5,(Doritos 12 oz.),(Bud Light 24 Pack),0.153745,0.241787,0.031537,0.205128,0.848384
22,(Coke 12 Pack),(Slim Jim),0.173456,0.204993,0.035480,0.204545,0.997815
9,(Lays Chips 12 oz.),(Bud Light 24 Pack),0.178712,0.241787,0.034166,0.191176,0.790681
7,(Hersheys Candy),(Bud Light 24 Pack),0.173456,0.241787,0.030223,0.174242,0.720644
1,(Coke 12 Pack),(Bud Light 24 Pack),0.173456,0.241787,0.030223,0.174242,0.720644
13,(Slim Jim),(Bud Light 24 Pack),0.204993,0.241787,0.035480,0.173077,0.715824


Many of our rules with a high confidence have Bud Light 24 Pack as the consequence. Bud Light 24 Pack is the most commonly purchased item, therefore we want to make sure that the high level of confidence isn't due to the high consequent support. In order to do this, we are going to focus on rules where the lift is greater than 1.05. When the lift is greater than 1, this means that the rule is useful because the consequent has an increased likelihood of purchase because the antecedent was purchased. The goal here is to cut out rules where the confidence is high but the antecent and consequent are actually independent. 

In [12]:
Rules = Rules[Rules.lift > 1.1]
Rules[['antecedents','consequents','support','confidence','lift']].sort_values('lift',ascending=False)

,antecedents,consequents,support,confidence,lift
2,(Bud Light 24 Pack),(Coke 20 oz),0.031537,0.130435,1.289102
3,(Coke 20 oz),(Bud Light 24 Pack),0.031537,0.311688,1.289102
11,(Miller Lite 24 Pack),(Bud Light 24 Pack),0.028909,0.301370,1.246426
10,(Bud Light 24 Pack),(Miller Lite 24 Pack),0.028909,0.119565,1.246426


This leaves the following as the most important patterns uncovered in the POS Data: 
- Bud Light 24 Pack -> Coke 20 oz: Customers who bought a Bud Light 24 Pack are 1.29 times more likely to buy a Coke 20 oz. Of customers who bought a Bud Light 24 Pack, 13% also bought a 20 oz Coke
- Coke 20 oz -> Bud Light 24 Pack: Customers who bought a 20 oz Coke are 1.29 times more likely to buy a Bud Light 24 Pack. Of customers who bought a 20 oz Coke, 31% also bought a Bud Light 24 Pack
- Miller Lite 24 Pack -> Bud Light 24 Pack: Customers who bought a Miller Lite 24 Pack are 1.25 times more likely to buy a Bud Light 24 Pack. Of customers who bought a Miller Lite 24 Pack, 30% also bought a Bud Light 24 Pack
- Bud Light 24 Pack -> Miller Lite 24 Pack: Customers who bought a Bud Light 24 Pack are 1.25 times more likely to buy a Miller Lite 24 Pack. Of customers who bought a Bud Light 24 Pack , 12% also bought a Miller Lite 24 Pack 

The retailers could use these rules to come up with cross-sell promotions. This could increase revenue by helping decrease the number of transactions that only contain one product. They also could use this information to help inform product placement within the store. By placing items likely to be purchased together near each other, customers may be more likely to grab a second related item if they see it. 